In [104]:
import numpy as np
import math
import random

from keras import Sequential
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.activations import relu, linear

import sys
import gym
import pandas as pd


In [105]:
class omniCopter():
    def __init__(self):
        self.running = True
        self.m = 1
        self.Ixx = 0.2
        self.l = 0.2
        
    def calcDerivatives(self,x,act,xd):
        # Extract the actions
        Tr = act[0]
        phird = act[1]
        Tl = act[2]
        phild = act[3]
        # Calculate the rotor forces in earth axes
        # Create the tilting rotor direction cosines
        phir = x[6]
        phil = x[7]
        Cpr_b = np.array([[1,0,0],
                          [0,math.cos(phir),-math.sin(phir)],
                          [0,math.sin(phir),math.cos(phir)]])
        Cpl_b = np.array([[1,0,0],
                          [0,math.cos(phil),-math.sin(phil)],
                          [0,math.sin(phil),math.cos(phil)]])
        Tvr = np.array([[0.0],[0.0],-Tr],dtype=object)
        Tvl = np.array([[0.0],[0.0],-Tl],dtype=object)
        Fr_b = Cpr_b @ Tvr
        Fl_b = Cpl_b @ Tvl
        # Now the body to NED axes
        phi = x[4]
        Cb_e = np.array([[1,0,0],
                          [0,math.cos(phi),-math.sin(phi)],
                          [0,math.sin(phi),math.cos(phi)]])
        # Then,
        Fr_e = Cb_e @ Fr_b
        Fl_e = Cb_e @ Fl_b
        # Total forces acting on the body are then,
        g = 10
        F = Fr_e + Fl_e + Cb_e @ np.array([[0],[0],[self.m * g]],dtype=object)
        # Now the moments. First transgform the moment arms into NED axes
        r_cg_pr_e = Cb_e @ np.array([[0],[self.l],[0]],dtype=object)
        r_cg_pl_e = Cb_e @ np.array([[0],[-self.l],[0]],dtype=object)
        # Now calculate the torque vector
        Tq = np.cross(np.transpose(r_cg_pr_e),np.transpose(Fr_e)) \
           + np.cross(np.transpose(r_cg_pl_e),np.transpose(Fl_e)) 
        #
        # With the forces and moments found, we can compute the linear and 
        # angular accelerations.
        ydd = F[1][0] / self.m
        zdd = F[2][0] / self.m
        phidd = Tq[0][1] / self.Ixx
        # Return the derivative vectors
        xd[0] = x[1]
        xd[1] = ydd
        xd[2] = x[3]
        xd[3] = zdd
        xd[4] = x[5]
        xd[5] = phidd
        xd[6] = phird
        xd[7] = phild
        return xd
        
    def runningStatus(self,s):
        self.running = s
        return self.running

In [106]:
class LanderEnv():
    def __init__(self):
        self.oc = omniCopter()
        
        self.ylim = [-5, 5]
        self.zlim = [-10, 0]
        
        self.tf = 2
      
    def render(self):
        plt.plot(self.yplt,self.zplt,label='omnicopter')
        plt.xlabel(r'$y$-axis')
        plt.ylabel(r'$-z$-axis')
        plt.axis([-5,5,0,10])
        plt.show()
        
    def reset(self):
        self.yplt = np.array([])
        self.zplt = []
        self.phiplt = []
        self.tplt = []
        
        y0 = random.randint(-3, 3)
        z0 = -9
        phi0 = random.uniform(-0.75, 0.75)
        self.current_state = np.array([[y0],[0.0],[z0],[0.0],[phi0],[0.0],[0.0],[0.0]])
        
        self.t = np.array([0.0])
        self.reward = 0
        
        return self.current_state.reshape(1,8)
    
    def get_reward(self, action, done, weights=[1,1,1,0.05,1,0.0015,1]):
        z_reward = self.current_state[2][0] #Z axis
        y_reward = -self.yplt.std() #Y axis
        p_reward = -np.abs(self.current_state[4][0] * 10) #Phi 
        t_reward = -len(self.tplt) #time
        a_reward = -np.abs(action[0][0]) -np.abs(action[2][0]) #Fuel(thrust)
        v_reward = -(self.current_state[1][0] + self.current_state[3][0] \
                     + self.current_state[5][0]) * ((20 + self.current_state[2][0])**2) #Velocity
        l_reward = 100 if -self.current_state[2][0] < 0.05 else 0
        total_reward = weights[0] * z_reward + weights[1] * y_reward + weights[2] * p_reward + \
                        weights[3] * t_reward + weights[4] * a_reward + weights[5] * v_reward + weights[6] * l_reward
        if done and not -self.current_state[2][0] < 0.05: total_reward -= 100
        return total_reward, [z_reward, y_reward, p_reward, t_reward, a_reward, v_reward, l_reward, total_reward]
        
    
    def step(self, action):
        dt = np.array([0.01])
        xd = np.zeros((8,1))
        action[0] = (action[0] + 1)*5
        action[2] = (action[2] + 1)*5
        action = action.reshape(4,1)
        xd = self.oc.calcDerivatives(self.current_state, action, xd)
        
        self.current_state += xd*dt
        
        self.t += dt
        
        self.yplt = np.append(self.yplt,self.current_state[0])
        self.zplt = np.append(self.zplt,-self.current_state[2])
        
        
        self.phiplt = np.append(self.phiplt,self.current_state[4])
        self.tplt = np.append(self.tplt,self.t)
        
        valid = (self.current_state[0][0] > self.ylim[0]) and (self.current_state[0][0] < self.ylim[1]) \
                and (self.current_state[2][0] > self.zlim[0]) and (self.current_state[2][0] < self.zlim[1])
        running = self.oc.runningStatus(valid and self.t<self.tf)
        
        self.reward, reward_breakdown = self.get_reward(action, not running)
        
        info = {'graphs': {'y':self.yplt, 'z': self.zplt, 'phi': self.phiplt, 't': self.tplt},
                'reward_breakdown': reward_breakdown}
        
        return self.current_state.reshape(1,8), self.reward, not running, info

In [96]:
env = LanderEnv()

In [97]:

import gym
# Ornstein-Ulhenbeck Process
# Taken from #https://github.com/vitchyr/rlkit/blob/master/rlkit/exploration_strategies/ou_strategy.py
class OUNoise(object):
    def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
        self.mu           = mu
        self.theta        = theta
        self.sigma        = max_sigma
        self.max_sigma    = max_sigma
        self.min_sigma    = min_sigma
        self.decay_period = decay_period
        self.action_dim   = 4
        self.low          = -1
        self.high         = 1
        self.reset()
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def evolve_state(self):
        x  = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state
    
    def get_action(self, action, t=0):
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
        return np.clip(action + ou_state, self.low, self.high)


# https://github.com/openai/gym/blob/master/gym/core.py
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """
    def _action(self, action):
        act_k = 1.
        act_b = 0.
        return act_k * action + act_b

    def _reverse_action(self, action):
        act_k_inv = 1.
        act_b = 0
        return act_k_inv * (action - act_b)
        

class Memory:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)
    
    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)
        
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

    def __len__(self):
        return len(self.buffer)

In [98]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.autograd
from torch.autograd import Variable

class Critic(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, state, action):
        """
        Params state and actions are torch tensors
        """
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x

class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 3e-4):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, state):
        """
        Param state is a torch tensor
        """
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = torch.tanh(self.linear3(x))

        return x

In [99]:
import torch
import torch.autograd
import torch.optim as optim
import torch.nn as nn

class DDPGagent:
    def __init__(self, env, hidden_size=256, actor_learning_rate=1e-4, critic_learning_rate=1e-3, gamma=0.99, tau=1e-2, max_memory_size=50000):
        # Params
        self.num_states = 8
        self.num_actions = 4
        self.gamma = gamma
        self.tau = tau

        # Networks
        self.actor = Actor(self.num_states, hidden_size, self.num_actions)
        self.actor_target = Actor(self.num_states, hidden_size, self.num_actions)
        self.critic = Critic(self.num_states + self.num_actions, hidden_size, self.num_actions)
        self.critic_target = Critic(self.num_states + self.num_actions, hidden_size, self.num_actions)

        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data)

        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)
        
        # Training
        self.memory = Memory(max_memory_size)        
        self.critic_criterion  = nn.MSELoss()
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)
    
    def get_action(self, state):
        state = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action = self.actor.forward(state)
        action = action.detach().numpy()[0,0]
        return action
    
    def update(self, batch_size):
        states, actions, rewards, next_states, _ = self.memory.sample(batch_size)
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
    
        # Critic loss 
        Qvals = self.critic.forward(states, actions)
        next_actions = self.actor_target.forward(next_states)
        next_Q = self.critic_target.forward(next_states, next_actions.detach())
        Qprime = rewards + self.gamma * next_Q
        critic_loss = self.critic_criterion(Qvals, Qprime)

        # Actor loss
        policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean()
        
        # update networks
        self.actor_optimizer.zero_grad()
        policy_loss.backward()
        self.actor_optimizer.step()

        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()

        # update target networks 
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))


In [101]:
print(action.shape, state.shape)

(4,) (1, 8)


In [100]:
agent = DDPGagent(env)
noise = OUNoise(None)
batch_size = 128
rewards = []
avg_rewards = []

for episode in range(50):
    state = env.reset()
    noise.reset()
    episode_reward = 0
    
    for step in range(500):
        action = agent.get_action(state)
        action = noise.get_action(action, step)
        new_state, reward, done, _ = env.step(action) 
        agent.memory.push(state, action, reward, new_state, done)
        
        if len(agent.memory) > batch_size:
            agent.update(batch_size)        
        
        state = new_state
        episode_reward += reward

        if done:
            sys.stdout.write("episode: {}, reward: {}, average _reward: {} \n".format(episode, np.round(episode_reward, decimals=2), np.mean(rewards[-10:])))
            break

    rewards.append(episode_reward)
    avg_rewards.append(np.mean(rewards[-10:]))
    

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at C:\Users\builder\AppData\Local\Temp\pip-req-build-e5c8dddg\aten\src\TH/generic/THTensor.cpp:603

In [28]:
plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

NameError: name 'rewards' is not defined